### this is used to construct the neural network for img registration
### the struct model I used is the HomographyNet

1 flow the very method in img registration<br>
2 use COCO2017 for further training<br>
3 ***in case the train test val are based in image set, so as we have to rebuilt the img set for the project***<br>
4 there is suggested that use the map to registration<br>
5 us the matrix for registration as result and so as the MSE method in matrix as the loss of further training<br>
6 as for the testing, use COCO2017 which we haven't used in training set<br>

In [1]:
import torch.nn as nn
import numpy as np

class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=2,out_channels=64,kernel_size=3,stride=1,padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64))
        self.layer2 = nn.Sequential(nn.Conv2d(64,64,3,1,1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64),
                                    nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(nn.Conv2d(64,64,3,1,1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64))
        
        self.layer4 = nn.Sequential(nn.Conv2d(64,64,3,1,1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64),
                                    nn.MaxPool2d(2))
        self.layer5 = nn.Sequential(nn.Conv2d(64,128,3,1,1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(128))
        self.layer6 = nn.Sequential(nn.Conv2d(128,128,3,1,1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(128),
                                    nn.MaxPool2d(2))
        self.layer7 = nn.Sequential(nn.Conv2d(128,128,3,1,1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(128))
        self.layer8 = nn.Sequential(nn.Conv2d(128,128,3,1,1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(128))
        self.fc1 = nn.Linear(128*16*16,1024,bias=True)
        self.fc2 = nn.Linear(1024,8,bias=True)
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = out.contiguous().view(-1,128*16*16)
        # keep the sequence memeory in case not be matched
        out = self.fc1(out)
        out = self.fc2(out)
        return out
# images = torch.randn(1,2,128,128).cuda()
# model = Model().cuda()
# summary(model,(2,128,128))
# outputs = model.forward(images)
# print(outputs)

### for training set set up and test set set up
### the method below are for model training and test
### a method of the adjust img with H_four_point
### return the batch of each img set, the return format is [(img1,img2)(four minus point)]

1 **the great ever change mathod in the matrix manufacture**<br>
2 use four point randomly to change with matching the maps<br>
3 then we return the map as it with a matrix which represent the adjust method<br>
4 use COCO2017 to settle the training and testing set<br>

In [31]:
from matplotlib import pyplot as plt
import cv2
import random
from numpy.linalg import inv
import numpy as np
import os

def ImagePreProcessing(image,path):
    img = cv2.imread(path+"/%s"%image,0)
    img = cv2.resize(img,(320,240))
    #make it small and easy to train
    
    rho = 32
    patch_size = 128
    top_point = (32,32)
    left_point = (patch_size+32,32)
    bottom_point = (patch_size+32,patch_size+32)
    right_point = (32,patch_size+32)
    test_image = img.copy()
    four_points = [top_point,left_point,bottom_point,right_point]
    
    perturbed_four_points = []
    for point in four_points:
        perturbed_four_points.append((point[0] + random.randint(-rho,rho), point[1]+random.randint(-rho,rho)))
        # move four point to change the map
    H = cv2.getPerspectiveTransform(np.float32(four_points),np.float32(perturbed_four_points))
    # use the four point to get the matrix
    H_inverse = inv(H)
    # does it usefu, it may not in that necessary
    warped_image = cv2.warpPerspective(img,H_inverse, (320,240))
    # use matrix to get the image adjusted
    annotated_warp_image = warped_image.copy()
    
    Ip1 = test_image[top_point[1]:bottom_point[1],top_point[0]:bottom_point[0]]
    # which is the source img
    Ip2 = warped_image[top_point[1]:bottom_point[1],top_point[0]:bottom_point[0]]
    # which is the goal img             
    training_image = np.dstack((Ip1, Ip2))
    # in dimension we stack two matrix map as one
    H_four_points = np.subtract(np.array(perturbed_four_points), np.array(four_points))
    # each point in matrix just match and minus each point in another matrix
    datum = (training_image, H_four_points)
    # make them as a real batch of training, [img1,img2],[minus matrix]
    return datum
    
def savedata(path):
    lst = os.listdir(path)
    if not os.path.exists(path+"processed/"):
        os.makedirs(path+"processed/")
    new_path = path+"processed/"
    for i in lst:
        np.save(new_path+"%s"%i[0:12],ImagePreProcessing(i,path))
        # in order to remove jpg, then use i[0:12]
savedata("./test")
savedata("./train")
savedata("./val")

D:\EssentialSoftware\python\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


### the method to get img from processed img Dataset
### List are below
1 ./trainprocessed folder for the training dataset<br>
2 ./testprocessed folder for the testing dataset<br>
3 ./valprocessed folder for the validation dataset<br>

In [2]:
import numpy as np
import os
class Cocodataset():
    def __init__(self,path):
        X = ()
        Y = ()
        lst = os.listdir(path)
        it = 0
        for i in lst:
            array = np.load(path+"%s"%i,allow_pickle=True)
            x = torch.from_numpy((array[0].astype(float)-127.5)/127.5)
            # transfer the numpy matrix to tensor and both of them share the same memory
            X = X+(x,)
            y = torch.from_numpy(array[1].astype(float)/32.)
            Y = Y+(y,)
            it += 1
        self.len = it
        self.X_data = X
        self.Y_data = Y
    def __getitem__(self,index):
        return self.X_data[index],self.Y_data[index]
    def __len__(self):
        return self.len
            

### the training method are below
### use MSE as the Loss function

### prepare the Dataset

In [4]:
import torch.nn as nn
import torch.nn as func
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

train_path = "./trainprocessed/"
validation_path = "./valprocessed/"

TrainingData = Cocodataset(train_path)
ValidationData = Cocodataset(validation_path)

print("the Training Dataset are:",TrainingData.len)
print("the Validation Dataset are:",ValidationData.len)



cuda
the Training Dataset are: 1787
the Validation Dataset are: 68


### Then we go for training the Dataset
#### iteration and training then get the best parameters

In [34]:
import torch
import torch.nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchsummary import summary
torch.cuda.empty_cache()
batch_size = 16
TrainLoader = DataLoader(TrainingData,batch_size)

criterion = nn.MSELoss()
num_samples = TrainingData.len
total_iteration = 5000
steps_per_epoch = num_samples / batch_size

epochs = int(total_iteration / steps_per_epoch)

model = Model().to(device)
summary(model,(2,128,128))
optimizer = optim.SGD(model.parameters(),lr=0.005,momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer,step_size=30000,gamma=0.1)
for epoch in range(epochs):
    for i,(images,target) in enumerate(TrainLoader):
        optimizer.zero_grad() 
        images = images.to(device)
        target = target.to(device)
        images = images.permute(0,3,1,2).float()
        target = target.float()
        outputs = model.forward(images)
        # print(outputs.size())
        # print(target.view(-1,8).size())
        loss = criterion(outputs,target.view(-1,8))
        loss.backward()
        optimizer.step()
        scheduler.step()
        if (i+1) % len(TrainLoader) == 0:
            print('Train Epoch: [{}/{}] [{}/{} ({:.0f}%)]\Mean Squared Error: {:.6f}'.format(
                epoch+1,epochs, i , len(TrainLoader),
                100. * i / len(TrainLoader), loss))
        
state = {'epoch': epochs, 'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict() }
torch.save(state, './DeepHomographyNet.pth')
print("model saved!")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           1,216
              ReLU-2         [-1, 64, 128, 128]               0
       BatchNorm2d-3         [-1, 64, 128, 128]             128
            Conv2d-4         [-1, 64, 128, 128]          36,928
              ReLU-5         [-1, 64, 128, 128]               0
       BatchNorm2d-6         [-1, 64, 128, 128]             128
         MaxPool2d-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,928
              ReLU-9           [-1, 64, 64, 64]               0
      BatchNorm2d-10           [-1, 64, 64, 64]             128
           Conv2d-11           [-1, 64, 64, 64]          36,928
             ReLU-12           [-1, 64, 64, 64]               0
      BatchNorm2d-13           [-1, 64, 64, 64]             128
        MaxPool2d-14           [-1, 64,

### for eval the model

In [6]:
import torch
import torch.nn
from torch.utils.data import DataLoader
criterion = nn.MSELoss()
batch_size = 32
ValidationLoader =DataLoader(ValidationData,batch_size)
model = Model().cuda()

# criterion = nn.MSELoss()
# from torchsummary import summary
# summary(model,(2,128,128)) 
# must formal
model.load_state_dict(torch.load('DeepHomographyNet.pth')['state_dict'])
# print(model)

print(model)
model.eval()

with torch.no_grad():
    for i,(images,target) in enumerate(ValidationLoader):
        images = images.to(device)
        target = target.to(device)
        images = images.permute(0,3,1,2).float()
        target = target.float()
        outputs = model(images)
        loss = criterion(outputs,target.view(-1,8))
        print(loss)
    # print("\Mean Squared Error: {:.6f}".format(np.mean(loss)))

Model(
  (layer1): Sequential(
    (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

##### Revise the Image

Repair it
the image which is aim to fix, by which, we can find its points can be find as a stable one

In [7]:
%matplotlib qt5
import numpy as np
import torch
from torch.utils.data import DataLoader
import cv2
from numpy.linalg import inv

test_path = "./testprocessedfromTrain/"
TestData = Cocodataset(test_path)
print("the TestData set are",TestData.len)

batch_size = 1
TestLoder = DataLoader(TestData,batch_size)
model = Model().cuda()
model.load_state_dict(torch.load("DeepHomographyNet.pth")["state_dict"])

patch_size = 128
top_point = (0,0)
left_point = (patch_size,0)
bottom_point = (patch_size,patch_size)
right_point = (0,patch_size)
four_points = [top_point,left_point,bottom_point,right_point]

with torch.no_grad():
    for i,(images,target) in enumerate(TestLoder):
        show = images[0]
        images = images.cuda()
        images = images.permute(0,3,1,2).float()
        outputs = model(images)
        outputs = outputs.view(-1,2)
        outputs = outputs.cpu()
        outputs = outputs.numpy()
        perturbed_four_points = np.add(np.array(four_points).astype(float), np.array(outputs).astype(float)*32.)
        H = cv2.getPerspectiveTransform(np.float32(four_points),np.float32(perturbed_four_points))
        H_inverse = inv(H)
        show = show.cpu()
        show = show.numpy()
        show = show*127.5+127.5
        show = show.astype("uint8")
        warped_image = cv2.warpPerspective(show[:,:,1],H,(320,240))
        print(show)
        cv2.imshow("source_image",np.array(show[:,:,0]))
        cv2.imshow("destin_image",np.array(show[:,:,1]))
        cv2.imshow("modify",warped_image)
        if cv2.waitKey(0) ==27:
            cv2.destroyAllWindows()
            break
# path = "./testprocessed"
# lst = os.listdir(path)
# for i in lst:
#     array = np.load(path+"/%s"%i,allow_pickle=True)
#     X = array[0]
#     print(X)
    # cv2.imshow("destination",np.array(X[:,:,0]))
    # cv2.imshow("source",np.array(X[:,:,1]))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()





the TestData set are 17
[[[153 140]
  [155 139]
  [162 139]
  ...
  [153 133]
  [153 133]
  [152 133]]

 [[158 139]
  [158 140]
  [157 139]
  ...
  [158 134]
  [162 134]
  [157 133]]

 [[157 137]
  [159 138]
  [158 138]
  ...
  [151 135]
  [127 135]
  [120 135]]

 ...

 [[ 37 142]
  [ 36 141]
  [ 36 140]
  ...
  [ 93  54]
  [105  54]
  [123  55]]

 [[128 141]
  [130 141]
  [132 141]
  ...
  [ 73  59]
  [121  58]
  [125  59]]

 [[148 140]
  [148 140]
  [148 139]
  ...
  [ 63  62]
  [131  62]
  [124  64]]]
[[[159 133]
  [ 91 106]
  [ 89  81]
  ...
  [227 164]
  [232 187]
  [226 212]]

 [[175 122]
  [161 131]
  [ 99 131]
  ...
  [222 176]
  [203 168]
  [216 156]]

 [[118 117]
  [ 78 144]
  [178 128]
  ...
  [139 191]
  [188 184]
  [166 175]]

 ...

 [[121 187]
  [118 188]
  [113 178]
  ...
  [163 238]
  [206 232]
  [167 231]]

 [[132 196]
  [121 191]
  [115 190]
  ...
  [212 241]
  [217 235]
  [192 230]]

 [[127 194]
  [111 193]
  [123 182]
  ...
  [223 237]
  [211 237]
  [163 232]]]
[[[1

KeyboardInterrupt: 

### for test the basic method that is needed

### see how the dataset had been settled

In [37]:
# %matplotlib qt5
# from matplotlib import pyplot as plt
# import cv2
# import random
# import torch
# from numpy.linalg import inv
# import numpy as np

# img = cv2.imread("2.jpg",0)
# img = cv2.resize(img,(320,240))

# rho          = 32
# patch_size   = 128
# top_point    = (32,32)
# left_point   = (patch_size+32, 32)
# bottom_point = (patch_size+32, patch_size+32)
# right_point  = (32, patch_size+32)
# test_image = img.copy()
# four_points = [top_point, left_point, bottom_point, right_point]

# perturbed_four_points = []
# for point in four_points:
#     perturbed_four_points.append((point[0] + random.randint(-rho,rho),point[1]+random.randint(-rho,rho)))
#     print((point[0] + random.randint(-rho,rho),point[1]+random.randint(-rho,rho)))
#     # move four point to change the map
# H = cv2.getPerspectiveTransform(np.float32(four_points),np.float32(perturbed_four_points))
# # use the four point to get the matrix
# H_inverse = inv(H)
# # does it usefu, it may not in that necessary
# warped_iamge = cv2.warpPerspective(img,H_inverse,(320,240))
# # use matrix to get the image adjusted
# Ip1 = test_image[top_point[1]:bottom_point[1],top_point[0]:bottom_point[0]]
# # which is the source img
# Ip2 = warped_iamge[top_point[1]:bottom_point[1],top_point[0]:bottom_point[0]]
# # which is the goal img                                        
# training_image = np.dstack((Ip1,Ip2))
# # in dimension we stack two matrix map as one
# H_four_points = np.subtract(np.array(perturbed_four_points),np.array(four_points))
# # each point in matrix just match and minus each point in another matrix
# datum = (training_image,H_four_points)
# # make them as a real batch of training, [img1,img2],[minus matrix]
# cv2.imshow("source",training_image[:,:,0])
# cv2.imshow("dest",training_image[:,:,1])
# backimg = cv2.warpPerspective(training_image[:,:,1],H,(320,240))
# cv2.imshow("back",backimg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

(41, 46)
(150, 17)
(164, 149)
(25, 188)
